In [1]:
import requests
import json
import pandas as pd
import plotly.express as px
from datetime import datetime
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import sys

In [2]:
# Dokumenteneinstellung
# DataFrame Options
np.set_printoptions(threshold=sys.maxsize)
pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 1000)

In [3]:
# Base URL for the OpenSenseMap API
base_url = "https://api.opensensemap.org/boxes"
# SenseBox ID
senseBoxId = "6645db6eeb5aad0007a6e4b6"
# List of Sensor IDs
sensorIds = [
    "6645db6eeb5aad0007a6e4b7",  # Replace with actual sensor IDs
    "6645db6eeb5aad0007a6e4b8",
    "6645db6eeb5aad0007a6e4b9",
    "6645db6eeb5aad0007a6e4ba"
]

# Parameters
params = {
    'format': 'json',            # or 'csv' if you prefer CSV format
    'from-date': '2024-05-18T09:00:00Z',
    #'to-date': datetime.now(),
    'download': 'true',          # This forces a download if format is 'csv'
    'outliers': 'mark',          # Optional: 'mark' or 'replace' if outlier detection is needed
    'outlier-window': 15,        # Optional: default is 15, can be 1-50
    'delimiter': 'comma'         # Optional: 'comma' or 'semicolon' for CSV
}

# Initialize an empty list to hold the data
all_data = []

# Loop through each sensor ID and retrieve data
for sensorId in sensorIds:
    # Construct the endpoint URL
    endpoint = f'{base_url}/{senseBoxId}/data/{sensorId}'
    
    # Send the GET request to the API
    response = requests.get(endpoint, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Convert response data to JSON
        data = response.json()
        # Add sensor ID to each measurement
        for measurement in data:
            measurement['Id'] = sensorId
            # Check for outliers and add an 'outlier' column
            if 'outlier' in measurement:
                measurement['outlier'] = measurement['outlier']
            else:
                measurement['outlier'] = None  # If no outlier info, set as None
        # Append data to the list
        all_data.extend(data)
    else:
        print(f'Error retrieving data for sensor {sensorId}: {response.status_code}')

# Convert the list of data to a DataFrame
sense_df = pd.DataFrame(all_data)
sense_df

,location,createdAt,value,isOutlier,Id,outlier
0,"[8.410042, 49.001548]",2024-05-21T12:37:06.513Z,17.14,False,6645db6eeb5aad0007a6e4b7,None
1,"[8.410042, 49.001548]",2024-05-21T12:36:06.424Z,17.19,False,6645db6eeb5aad0007a6e4b7,None
2,"[8.410042, 49.001548]",2024-05-21T12:35:06.255Z,17.04,False,6645db6eeb5aad0007a6e4b7,None
3,"[8.410042, 49.001548]",2024-05-21T12:34:06.213Z,16.79,False,6645db6eeb5aad0007a6e4b7,None
4,"[8.410042, 49.001548]",2024-05-21T12:33:06.278Z,16.64,False,6645db6eeb5aad0007a6e4b7,None
...,...,...,...,...,...,...
18111,"[8.410042, 49.001548]",2024-05-18T09:04:12.890Z,326.25,False,6645db6eeb5aad0007a6e4ba,None
18112,"[8.410042, 49.001548]",2024-05-18T09:03:12.845Z,326.25,False,6645db6eeb5aad0007a6e4ba,None
18113,"[8.410042, 49.001548]",2024-05-18T09:02:12.759Z,320.62,False,6645db6eeb5aad0007a6e4ba,None
18114,"[8.410042, 49.001548]",2024-05-18T09:01:12.694Z,320.62,False,6645db6eeb5aad0007a6e4ba,None


In [4]:
sense_df['createdAt'] = pd.to_datetime(sense_df['createdAt'])

sensor_info = {
    '6645db6eeb5aad0007a6e4b7': {'name': 'Temperatur', 'unit': '°C'},
    '6645db6eeb5aad0007a6e4b8': {'name': 'rel. Luftfeuchte', 'unit': '%'},
    '6645db6eeb5aad0007a6e4b9': {'name': 'Luftdruck', 'unit': 'hPa'},
    '6645db6eeb5aad0007a6e4ba': {'name': 'UV-Intensität', 'unit': 'μW/cm²'},
}

sense_df['sensor_name'] = sense_df['Id'].map(lambda x: sensor_info[x]['name'])
sense_df['unit'] = sense_df['Id'].map(lambda x: sensor_info[x]['unit'])

In [5]:
sense_df.dtypes

location                    object
createdAt      datetime64[ns, UTC]
value                       object
isOutlier                     bool
Id                          object
outlier                     object
sensor_name                 object
unit                        object
dtype: object

# Outlier

In [6]:
outliers_df = sense_df[sense_df['isOutlier'] == True]
outliers_df

,location,createdAt,value,isOutlier,Id,outlier,sensor_name,unit
31,"[8.410042, 49.001548]",2024-05-21 12:06:02.777000+00:00,16.15,True,6645db6eeb5aad0007a6e4b7,None,Temperatur,°C
32,"[8.410042, 49.001548]",2024-05-21 12:05:02.846000+00:00,16.09,True,6645db6eeb5aad0007a6e4b7,None,Temperatur,°C
33,"[8.410042, 49.001548]",2024-05-21 12:04:02.868000+00:00,16.07,True,6645db6eeb5aad0007a6e4b7,None,Temperatur,°C
35,"[8.410042, 49.001548]",2024-05-21 12:02:02.449000+00:00,16.11,True,6645db6eeb5aad0007a6e4b7,None,Temperatur,°C
50,"[8.410042, 49.001548]",2024-05-21 11:47:00.526000+00:00,16.52,True,6645db6eeb5aad0007a6e4b7,None,Temperatur,°C
...,...,...,...,...,...,...,...,...
18106,"[8.410042, 49.001548]",2024-05-18 09:09:13.536000+00:00,309.38,True,6645db6eeb5aad0007a6e4ba,None,UV-Intensität,μW/cm²
18107,"[8.410042, 49.001548]",2024-05-18 09:08:14.860000+00:00,309.38,True,6645db6eeb5aad0007a6e4ba,None,UV-Intensität,μW/cm²
18108,"[8.410042, 49.001548]",2024-05-18 09:07:13.300000+00:00,315.00,True,6645db6eeb5aad0007a6e4ba,None,UV-Intensität,μW/cm²
18109,"[8.410042, 49.001548]",2024-05-18 09:06:13.163000+00:00,326.25,True,6645db6eeb5aad0007a6e4ba,None,UV-Intensität,μW/cm²


In [7]:
num_outliers = sense_df['isOutlier'].sum()
print("Anzahl der Outlier im DataFrame:", num_outliers)

Anzahl der Outlier im DataFrame: 3980


# Anzahl Messwerte

In [8]:
num_measurements = sense_df.shape[0]
print("Anzahl der Messwerte im DataFrame:", num_measurements)

Anzahl der Messwerte im DataFrame: 18116


In [9]:
# Gesamtzahl der Messwerte
total_measurements = sense_df.shape[0]
# Prozentsatz der Outliers
percent_outliers = (num_outliers / total_measurements) * 100

print(f"Prozentsatz der Outliers im DataFrame: {percent_outliers:.2f}%")

Prozentsatz der Outliers im DataFrame: 21.97%


# Bereinigter Datensatz

In [10]:
df = sense_df[sense_df['isOutlier'] == False]
df

,location,createdAt,value,isOutlier,Id,outlier,sensor_name,unit
0,"[8.410042, 49.001548]",2024-05-21 12:37:06.513000+00:00,17.14,False,6645db6eeb5aad0007a6e4b7,None,Temperatur,°C
1,"[8.410042, 49.001548]",2024-05-21 12:36:06.424000+00:00,17.19,False,6645db6eeb5aad0007a6e4b7,None,Temperatur,°C
2,"[8.410042, 49.001548]",2024-05-21 12:35:06.255000+00:00,17.04,False,6645db6eeb5aad0007a6e4b7,None,Temperatur,°C
3,"[8.410042, 49.001548]",2024-05-21 12:34:06.213000+00:00,16.79,False,6645db6eeb5aad0007a6e4b7,None,Temperatur,°C
4,"[8.410042, 49.001548]",2024-05-21 12:33:06.278000+00:00,16.64,False,6645db6eeb5aad0007a6e4b7,None,Temperatur,°C
...,...,...,...,...,...,...,...,...
18111,"[8.410042, 49.001548]",2024-05-18 09:04:12.890000+00:00,326.25,False,6645db6eeb5aad0007a6e4ba,None,UV-Intensität,μW/cm²
18112,"[8.410042, 49.001548]",2024-05-18 09:03:12.845000+00:00,326.25,False,6645db6eeb5aad0007a6e4ba,None,UV-Intensität,μW/cm²
18113,"[8.410042, 49.001548]",2024-05-18 09:02:12.759000+00:00,320.62,False,6645db6eeb5aad0007a6e4ba,None,UV-Intensität,μW/cm²
18114,"[8.410042, 49.001548]",2024-05-18 09:01:12.694000+00:00,320.62,False,6645db6eeb5aad0007a6e4ba,None,UV-Intensität,μW/cm²


# Daten in eine Zeile schreiben 

In [11]:
opensense_df = df.pivot_table(
    index='createdAt', 
    columns='sensor_name', 
    values='value'
).reset_index()

opensense_df

sensor_name,createdAt,Luftdruck,Temperatur,UV-Intensität,rel. Luftfeuchte
0,2024-05-18 09:00:12.464000+00:00,998.53,18.27,315.0,NaN
1,2024-05-18 09:01:12.694000+00:00,998.49,18.36,320.62,NaN
2,2024-05-18 09:02:12.759000+00:00,998.52,18.3,320.62,NaN
3,2024-05-18 09:03:12.845000+00:00,998.5,18.5,326.25,NaN
4,2024-05-18 09:04:12.890000+00:00,998.49,18.52,326.25,69.38
...,...,...,...,...,...
4493,2024-05-21 12:33:06.278000+00:00,994.12,16.64,118.12,87.44
4494,2024-05-21 12:34:06.213000+00:00,994.13,16.79,118.12,87.19
4495,2024-05-21 12:35:06.255000+00:00,994.12,17.04,106.88,85.94
4496,2024-05-21 12:36:06.424000+00:00,994.16,17.19,112.5,85.54


# Analysen

In [12]:
opensense_df.corr()

sensor_name,createdAt,Luftdruck,Temperatur,UV-Intensität,rel. Luftfeuchte
sensor_name,,,,,
createdAt,1.000000,-0.792579,-0.005478,-0.275003,0.231663
Luftdruck,-0.792579,1.000000,-0.231136,0.275745,-0.004023
Temperatur,-0.005478,-0.231136,1.000000,0.653513,-0.935670
UV-Intensität,-0.275003,0.275745,0.653513,1.000000,-0.674449
rel. Luftfeuchte,0.231663,-0.004023,-0.935670,-0.674449,1.000000


In [13]:
opensense_df.to_csv('sensor_data.csv', index=False)

In [14]:
opensense_df

sensor_name,createdAt,Luftdruck,Temperatur,UV-Intensität,rel. Luftfeuchte
0,2024-05-18 09:00:12.464000+00:00,998.53,18.27,315.0,NaN
1,2024-05-18 09:01:12.694000+00:00,998.49,18.36,320.62,NaN
2,2024-05-18 09:02:12.759000+00:00,998.52,18.3,320.62,NaN
3,2024-05-18 09:03:12.845000+00:00,998.5,18.5,326.25,NaN
4,2024-05-18 09:04:12.890000+00:00,998.49,18.52,326.25,69.38
...,...,...,...,...,...
4493,2024-05-21 12:33:06.278000+00:00,994.12,16.64,118.12,87.44
4494,2024-05-21 12:34:06.213000+00:00,994.13,16.79,118.12,87.19
4495,2024-05-21 12:35:06.255000+00:00,994.12,17.04,106.88,85.94
4496,2024-05-21 12:36:06.424000+00:00,994.16,17.19,112.5,85.54


# Resample

# Visualisierung

In [15]:
fig = px.line(opensense_df, x='createdAt', y='Temperatur',title='Temperaturverlauf(°C) über Zeit')

# Achsentitel hinzufügen
fig.update_xaxes(title_text='Messzeitpunkte')
fig.update_yaxes(title_text='Temperatur (°C)')

# Achsenbeschriftungen verbessern
fig.update_xaxes(tickangle=45, tickformat='%Y-%m-%d')

# Gitterlinien hinzufügen
fig.update_layout(xaxis=dict(showgrid=True, gridwidth=1, gridcolor='LightGrey'),
                  yaxis=dict(showgrid=True, gridwidth=1, gridcolor='LightGrey'))

# Bildunterschrift unterhalb der Achsenbeschriftungen hinzufügen
fig.update_layout(annotations=[dict(xref='paper', yref='paper', x=0.5, y=0.0, showarrow=False, text="Hinweis: Ausreißer wurden entfernt")])

fig.show()


In [16]:
# Temperatur und Luffeuchte
fig = px.line(opensense_df, x='createdAt', y=['Temperatur'], title='Temperatur und Luftfeuchte über Zeit')
fig.add_scatter(x=opensense_df['createdAt'], y=opensense_df['rel. Luftfeuchte'], mode='lines', name='rel. Luftfeuchte', yaxis='y2')

# Achsentitel hinzufügen
fig.update_xaxes(title_text='Messzeitpunkte')
# Achsenbeschriftungen verbessern
fig.update_xaxes(tickangle=45, tickformat='%Y-%m-%d')


fig.update_layout(yaxis=dict(title='Temperatur (°C)'), yaxis2=dict(title='rel. Luftfeuchte (%)', overlaying='y', side='right'))
fig.update_layout(annotations=[dict(xref='paper', yref='paper', x=0.5, y=0.0, showarrow=False, text="Hinweis: Ausreißer wurden entfernt")])
fig.show()

In [17]:
# Temperatur und UV-Index
fig = px.line(opensense_df, x='createdAt', y=['Temperatur'], title='Temperatur und UV-Intensität über Zeit')
fig.add_scatter(x=opensense_df['createdAt'], y=opensense_df['UV-Intensität'], mode='lines', name='UV-Intensität', yaxis='y2')

# Achsentitel hinzufügen
fig.update_xaxes(title_text='Messzeitpunkte')
# Achsenbeschriftungen verbessern
fig.update_xaxes(tickangle=45, tickformat='%Y-%m-%d')

fig.update_layout(yaxis=dict(title='Temperatur (°C)'), yaxis2=dict(title='UV-Intensität (μW/cm²)', overlaying='y', side='right'))
fig.update_layout(annotations=[dict(xref='paper', yref='paper', x=0.5, y=0.0, showarrow=False, text="Hinweis: Ausreißer wurden entfernt")])
fig.show()

In [18]:
# Temperatur und Luffeuchte
fig = px.line(opensense_df, x='createdAt', y=['Luftdruck'], title='Luftdruck und Luftfeuchte über Zeit')
fig.add_scatter(x=opensense_df['createdAt'], y=opensense_df['rel. Luftfeuchte'], mode='lines', name='rel. Luftfeuchte', yaxis='y2')

# Achsentitel hinzufügen
fig.update_xaxes(title_text='Messzeitpunkte')
# Achsenbeschriftungen verbessern
fig.update_xaxes(tickangle=45, tickformat='%Y-%m-%d')


fig.update_layout(yaxis=dict(title='Luftdruck (hPa)'), yaxis2=dict(title='rel. Luftfeuchte (%)', overlaying='y', side='right'))
fig.update_layout(annotations=[dict(xref='paper', yref='paper', x=0.5, y=0.0, showarrow=False, text="Hinweis: Ausreißer wurden entfernt")])
fig.show()

# Dashboard

In [25]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import requests

# Base URL for the OpenSenseMap API
base_url = "https://api.opensensemap.org/boxes"
# SenseBox ID
senseBoxId = "6645db6eeb5aad0007a6e4b6"
# List of Sensor IDs
sensorIds = [
    "6645db6eeb5aad0007a6e4b7",
    "6645db6eeb5aad0007a6e4b8",
    "6645db6eeb5aad0007a6e4b9",
    "6645db6eeb5aad0007a6e4ba"
]

# Parameters
params = {
    'format': 'json',
    'from-date': '2024-05-18T09:00:00Z',
    'download': 'false',
    'outliers': 'mark',
    'outlier-window': 15
}

sensor_info = {
    '6645db6eeb5aad0007a6e4b7': {'name': 'Temperatur', 'unit': '°C'},
    '6645db6eeb5aad0007a6e4b8': {'name': 'rel. Luftfeuchte', 'unit': '%'},
    '6645db6eeb5aad0007a6e4b9': {'name': 'Luftdruck', 'unit': 'hPa'},
    '6645db6eeb5aad0007a6e4ba': {'name': 'UV-Intensität', 'unit': 'μW/cm²'},
}

# Function to fetch data from OpenSenseMap API
def fetch_sensor_data(sensorId):
    endpoint = f'{base_url}/{senseBoxId}/data/{sensorId}'
    response = requests.get(endpoint, params=params)
    if response.status_code == 200:
        data = response.json()
        for measurement in data:
            measurement['Id'] = sensorId
            measurement['outlier'] = measurement.get('outlier', None)
        return data
    else:
        print(f'Error retrieving data for sensor {sensorId}: {response.status_code}')
        return []

# Initialize an empty list to hold the data
all_data = []

# Loop through each sensor ID and retrieve data
for sensorId in sensorIds:
    all_data.extend(fetch_sensor_data(sensorId))

# Convert the list of data to a DataFrame
sense_df = pd.DataFrame(all_data)
sense_df['createdAt'] = pd.to_datetime(sense_df['createdAt'])
sense_df['sensor_name'] = sense_df['Id'].map(lambda x: sensor_info[x]['name'])
sense_df['unit'] = sense_df['Id'].map(lambda x: sensor_info[x]['unit'])

# Filter out outliers
sense_df = sense_df[sense_df['outlier'] != True]

# Pivot the DataFrame for easier plotting
opensense_df = sense_df.pivot_table(
    index='createdAt', 
    columns='sensor_name', 
    values='value'
).reset_index()

# Anzahl der Messpunkte
num_measurements = len(sense_df)
# Anzahl der Fehlerhaften Messungen
num_outliers = sense_df['isOutlier'].sum()
# Prozentsatz der Outliers
percent_outliers = round((num_outliers / num_measurements) * 100,2)


# Function to create a figure
def create_figure(df, x, y, title, y_title, y2_title=None):
    fig = px.line(df, x=x, y=y, title=title)
    fig.update_xaxes(title_text='Messzeitpunkte')
    fig.update_xaxes(tickangle=45, tickformat='%Y-%m-%d')
    fig.update_layout(
        xaxis=dict(showgrid=True, gridwidth=1, gridcolor='LightGrey'),
        yaxis=dict(showgrid=True, gridwidth=1, gridcolor='LightGrey'),
        annotations=[dict(xref='paper', yref='paper', x=0.5, y=0.0, showarrow=False, text="Hinweis: Ausreißer wurden entfernt")]
    )
    if y2_title:
        fig.add_scatter(x=df[x], y=df[y2_title], mode='lines', name=y2_title, yaxis='y2')
        fig.update_layout(yaxis=dict(title=y_title), yaxis2=dict(title=y2_title, overlaying='y', side='right'))
    else:
        fig.update_yaxes(title_text=y_title)
    return fig

# Create the figures
temp_figure = create_figure(opensense_df, 'createdAt', 'Temperatur', 'Temperaturverlauf (°C) über Zeit', 'Temperatur (°C)')
temp_humid_figure = create_figure(opensense_df, 'createdAt', ['Temperatur'], 'Temperatur und rel. Luftfeuchte über Zeit', 'Temperatur (°C)', 'rel. Luftfeuchte')
temp_uv_figure = create_figure(opensense_df, 'createdAt', ['Temperatur'], 'Temperatur und UV-Intensität über Zeit', 'Temperatur (°C)', 'UV-Intensität')
pressure_humid_figure = create_figure(opensense_df, 'createdAt', ['Luftdruck'], 'Luftdruck und rel. Luftfeuchte über Zeit', 'Luftdruck (hPa)', 'rel. Luftfeuchte')

# Create the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    # Sidebar on the left side
    html.Div(
        id='side-panel',
        style={
            'width': '25%',  
            'position': 'fixed',  
            'top': 0,  # Align to the top
            'left': 0,  # Align to the left
            'bottom': 0,  # Align to the bottom
            'backgroundColor': '#f0f0f0', 
            'overflowY': 'scroll'  # Enable scrolling if content overflows
        },
        children=[
            html.Div([
                html.H2('Umweltmonitoring SS24 Projektarbeit'),
                html.P('Im Rahmen eines Fachpraktikums wurde eine OpenSenseBox zur Umweltüberwachung aufgebaut, um Daten wie Luftqualität und Temperatur zu erfassen und zu visualisieren.Die Echtzeitvisualisierung bietet Einblicke in lokale Umweltbedingungen.Ein weiteres Ziel ist die Entwicklung eines Modells zur Wettervorhersage basierend auf diesen Daten'),
                # Example image with adjusted size
                html.Img(
                    src='https://docs.sensebox.de/images/2020-10-16-opensensemap-faq/openSenseMap_github.png',
                    style={
                        'width': '200px',  
                        'height': '90px'
                    }
                ),
                html.P(f"Anzahl der Messpunkte: {num_measurements}"),
                html.P(f"Anzahl der Fehlerhaften Messungen: {num_outliers}"),
                html.P(f"Anteil der Fehlerhaften Messungen: {percent_outliers}%"),
                html.P("Sensoren:"),
                html.Ul([
                    html.Li(f"{sensor_info[sensor_id]['name']} ({sensor_info[sensor_id]['unit']})")
                    for sensor_id in sensorIds
                ]),
                html.P("Beispiel text"),
            ], style={'padding': '20px'}),
            html.Footer("Gruppe: Alexandru, Evelyn, Rafael", style={'textAlign': 'left', 'paddingLeft': '20px'})  
        ]
    ),
    
    # Main content on the right side
    html.Div([
        dbc.Container([
            dbc.Row([
                dbc.Col(dcc.Graph(figure=temp_figure), width=6),
                dbc.Col(dcc.Graph(figure=temp_humid_figure), width=6)
            ]),
            dbc.Row([
                dbc.Col(dcc.Graph(figure=temp_uv_figure), width=6),
                dbc.Col(dcc.Graph(figure=pressure_humid_figure), width=6)
            ])
        ], fluid=True)
    ], style={'marginLeft': '25%', 'padding': '10px'})  # Adjust left margin to make space for the sidebar
])

if __name__ == '__main__':
    app.run_server(debug=True)


# Datenbankverbidnung

In [31]:
createdb sense_data
psql -d sense_data -c "CREATE EXTENSION postgis;"


SyntaxError: invalid syntax (4068035281.py, line 1)

In [30]:
CREATE TABLE sensor_data (
    id SERIAL PRIMARY KEY,
    sensor_id VARCHAR(255),
    created_at TIMESTAMP,
    value DOUBLE PRECISION,
    outlier BOOLEAN,
    sensor_name VARCHAR(255),
    unit VARCHAR(255),
    geom GEOGRAPHY(Point, 4326)
);


SyntaxError: invalid syntax (1296043975.py, line 1)

In [32]:
import requests
import pandas as pd
import psycopg2
from datetime import datetime

# Datenbankverbindung herstellen
conn = psycopg2.connect(
    dbname="sense_data",
    user="postgres",
    password="test",  # Ersetzen Sie 'your_password' durch Ihr tatsächliches Passwort
    host="localhost"
)
cur = conn.cursor()

# Base URL für die OpenSenseMap API
base_url = "https://api.opensensemap.org/boxes"
senseBoxId = "6645db6eeb5aad0007a6e4b6"
sensorIds = [
    "6645db6eeb5aad0007a6e4b7",
    "6645db6eeb5aad0007a6e4b8",
    "6645db6eeb5aad0007a6e4b9",
    "6645db6eeb5aad0007a6e4ba"
]

params = {
    'format': 'json',
    'from-date': '2024-05-18T09:00:00Z',
    'download': 'true',
    'outliers': 'mark',
    'outlier-window': 15,
    'delimiter': 'comma'
}

sensor_info = {
    '6645db6eeb5aad0007a6e4b7': {'name': 'Temperatur', 'unit': '°C'},
    '6645db6eeb5aad0007a6e4b8': {'name': 'rel. Luftfeuchte', 'unit': '%'},
    '6645db6eeb5aad0007a6e4b9': {'name': 'Luftdruck', 'unit': 'hPa'},
    '6645db6eeb5aad0007a6e4ba': {'name': 'UV-Intensität', 'unit': 'μW/cm²'},
}

# Daten abrufen und in die Datenbank importieren
for sensorId in sensorIds:
    endpoint = f'{base_url}/{senseBoxId}/data/{sensorId}'
    response = requests.get(endpoint, params=params)
    if response.status_code == 200:
        data = response.json()
        for measurement in data:
            created_at = measurement['createdAt']
            value = measurement['value']
            outlier = measurement.get('outlier', None)
            sensor_name = sensor_info[sensorId]['name']
            unit = sensor_info[sensorId]['unit']
            # Hier fügen Sie die Geometrie hinzu (z.B. den Standort der SenseBox, dies ist ein Beispielwert)
            geom = 'SRID=4326;POINT(7.6285 51.9607)'  # Ersetzen Sie longitude und latitude durch tatsächliche Werte
            
            # SQL-Befehl zum Einfügen der Daten
            insert_query = """
                INSERT INTO sensor_data (sensor_id, created_at, value, outlier, sensor_name, unit, geom)
                VALUES (%s, %s, %s, %s, %s, %s, ST_GeogFromText(%s));
            """
            cur.execute(insert_query, (sensorId, created_at, value, outlier, sensor_name, unit, geom))

conn.commit()
cur.close()
conn.close()


In [35]:
# Verbindung zur Datenbank herstellen
conn = psycopg2.connect(
    dbname="sense_data",
    user="postgres",
    password="your_password",
    host="localhost"
)

# Cursor erstellen
cur = conn.cursor()

# SQL-Abfrage definieren
sql_query = "SELECT * FROM sensor_data WHERE sensor_id = %s;"

# Parameter für die Abfrage
sensor_id = '6645db6eeb5aad0007a6e4b7'

# Abfrage ausführen
cur.execute(sql_query, (sensor_id,))

# Ergebnisse abrufen
results = cur.fetchall()

# Ergebnisse anzeigen
for row in results:
    print(row)

# Cursor und Verbindung schließen
cur.close()
conn.close()

SyntaxError: invalid syntax (657934599.py, line 1)

In [36]:
conn = psycopg2.connect(
    dbname="sense_data",
    user="postgres",
    password="your_password",
    host="localhost"
)
query = "SELECT * FROM sensor_data;"
df = pd.read_sql_query(query, conn)

fig = px.scatter(df, x='created_at', y='value', color='sensor_name', title='Sensor Data over Time')
fig.show()


/var/folders/hl/g54whv_j0vd21tv5fjmrvw6m0000gn/T/ipykernel_61271/1948749746.py:13: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

